## Setup

In [ ]:
%load_ext sql

In [ ]:
import pandas as pd
from dotenv import find_dotenv
from dotenv import load_dotenv
from nanoid import generate
load_dotenv(find_dotenv())

In [ ]:
import google.auth
credentials, project_id = google.auth.default()

In [ ]:
import os
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def get_conn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASSWORD,
        db=DB_NAME
    )
    return conn

def get_pool():
    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=get_conn,
    )
    
    return pool

## Create Business Connection

In [ ]:
DB_USER = os.getenv("BUSINESS_USERNAME", "")
DB_PASSWORD = os.getenv("BUSINESS_PASSWORD", "")
DB_NAME = os.getenv("BUSINESS_NAME", "")
INSTANCE_CONNECTION_NAME = os.getenv("BUSINESS_CONNECTION", "")

business_pool = get_pool()
%sql business_pool --alias business_db
%sql business_db

In [ ]:
%%sql
select now();

## Create Warehouse Connection

In [ ]:
DB_USER = os.getenv("WAREHOUSE_USERNAME", "")
DB_PASSWORD = os.getenv("WAREHOUSE_PASSWORD", "")
DB_NAME = os.getenv("WAREHOUSE_NAME", "")
INSTANCE_CONNECTION_NAME = os.getenv("WAREHOUSE_CONNECTION", "")

warehouse_pool = get_pool()
%sql warehouse_pool --alias warehouse_db
%sql warehouse_db

In [ ]:
%%sql
select now();

## List Database Connections

In [ ]:
%sql --connections

## Query Business Database

In [ ]:
%sql business_db

In [ ]:
%%sql business_data <<
select * from business

In [ ]:
business_data

## Query Warehouse Database

In [ ]:
%sql warehouse_db

In [ ]:
%%sql colin_data <<
SELECT cn.corp_nme, c.corp_typ_cd, c.corp_num, c.bn_15
from colin.corporation c, colin.corp_name cn 
where c.corp_num = cn.corp_num
and cn.end_event_id is null
limit 5

In [ ]:
colin_df = colin_data.DataFrame()
colin_df

## Send data to Business AR for use

In [ ]:
colin_df['nano_id'] = colin_df.apply(lambda row: generate(), axis = 1)
colin_df

In [ ]:
colin_df.to_sql(name='business', con=business_pool, if_exists='append', index=False)

In [ ]:
%%sql
select * from business

## Hand crafted data

In [ ]:
business_df = pd.DataFrame({'id':[4],'legal_name':['cookie2'],'legal_type':['BC'], 'identifier':[ '0005063'],'tax_id':[None]})
business_df['nano_id'] = business_df.apply(lambda row: generate(), axis = 1)
business_df.to_sql(name='business', con=business_pool, if_exists='append', index=False)